In [1]:
import pandas as pd
import numpy as np
import json

In [5]:
resumes = []
with open('./resume.json') as f:
    for line in f.readlines():
            resumes.append(json.loads(line))

In [2]:
vac = []
with open('./vacancy.json') as f:
    for line in f.readlines():
            vac.append(json.loads(line))

# препроцессинг hard skills
текущая схема

    1) разбить токенайзером
    2) нормализовать слова
    3) удалить стопслова
    4) сделать сплит

In [2]:
import pymorphy2
import re
from nltk.corpus import stopwords
from nltk import word_tokenize

STOP_WORDS = stopwords.words('russian')
SPLIT_SYMS = '[()!,.:;\n]-/*'
morph = pymorphy2.MorphAnalyzer()    
NORMALIZE = lambda word: morph.parse(word)[0].normal_form # для ускорения работы
def preprocess(line):
    '''
    input: str для препроцессинга (resume['best'])
    output: list предпологаемых hard skills
    '''
    # токенайзер сплитанет по словам нормально (получше чем мы кривыми руками напишем)
    tokenized = word_tokenize(line.lower())
    
    #приводим слова к начальной форме
    tokenized = map(NORMALIZE, tokenized)
    
    # удаляем стопслова
    tokenized = [x for x in tokenized if x not in STOP_WORDS]
    
    #делаем сплит
    output = []                 # список хард скиллов
    prev_index = -1
    for index, i in enumerate(tokenized):
        if len(i) == 1 and i in SPLIT_SYMS:
            current_skill = tokenized[prev_index+1:index]
            prev_index = index
            if current_skill:   # не нужны пустые скиллы
                output.append(current_skill)
    return output

In [3]:
from tqdm import tqdm

In [71]:
hs = {}
for resume in tqdm(resumes):
    hs[resume['id']] = (preprocess(resume['best']))

100%|██████████| 128945/128945 [04:51<00:00, 442.01it/s]


In [ ]:
f = open('skills.json', 'w')
json.dump(hs, f)
f.close()

In [4]:
with open('./skills.json') as f:
    skills = (json.loads(f.readline()))

In [5]:
MY_STOPWORDS = (
        ['ms', 'microsoft', 'etc'] + ['2008', '2000', '2003', '2007', '2010', '2012', '2011', '2009',
       '2005', '2014', '2006', '2013']
    +['1', '82', '', 'ad', '7', '2', '3', '8', 'cc', '77',
 '81', '4', '83', '5', '80', 'ос', '6', '3d', '2d', '10',
'$', 'скс', 'сс', 'фсс', '98', '9']
+['тч', 'др', 'тд', 'впр', 'пр', 'тп', 'тк']
+['10$год', '5$год', '15$год', '2$год']
+['предприятие$8', 'предприятие$82']
)

In [6]:
s = []
for i in skills:
    for j in skills[i]:
        curr_skill = []
        for word in j:
            word = ''.join([c for c in word if str.isalnum(c)])
            if word not in MY_STOPWORDS:
                curr_skill.append(word)
        if curr_skill:
            s.append('$'.join(curr_skill))

In [7]:
a, b = np.unique(s, return_counts=True)

In [8]:
c = list(zip(a, b))

In [9]:
c = sorted(c, key=lambda x: x[1], reverse=True)

# кластеризация по levensthein

In [10]:
from levinshtein_distance import distance as levenshtein      
import numpy as np
from sklearn.cluster import DBSCAN, dbscan
SYNONYMS  = (
    [('1c', '1с8'), ('1с', '1с8'), ('$1с', '1с8'), ('1с8', '1с82'), ('1с$82', '1с8'),
     ('1с$8', '1с8'), ('1с$8', '1с$82'), ('1с$8', '1с$82'), ('1с', '1с82'), ('1с$8', '1с$8'),
     ('1c', '1с82'), ('1с', '1с$8'), ('$1с', '1с$8'), ('$1с', '1с'), ('1c', '1с$82'),
     ('$1с', '1с$82'), ('$1с', '1c'), ('$1с', '1с82'), ('1c', '1с$8'), ('1с$8', '1с82'), ('1с$82', '1с82'), ('1c', '1с'), ('1с', '1с$82'), ('c', 'с'), ('c$', 'с'), ('c', 'c$')]
+['1с$бухгалтерия' '1сбухгалтерия' '1c$бухгалтерия']
+[('exel', 'exсel'), ('excel', 'excell'), ('excell', 'exsel'), ('exel', 'exell'), ('exell', 'exsel'), ('exel', 'exsel'), ('excel', 'exсel'), ('excell', 'exсel'), ('exsel', 'exсel'), ('excell', 'exell'), ('excell', 'exel'), ('exell', 'exсel'), ('excel', 'exell'), ('excel', 'exel'), ('excel', 'exsel')]
+[('$консультант$$', 'консультант'), ('$консультант$', 'консультант$'), ('$консультант$', '$консультант$$'), ('$консультант$$', 'консультант$'), ('$консультант$', 'консультант'), ('консультант', 'консультант$')]
+[('1с$предприятие', '1спредприятие'), ('1c$предприятие', '1с$предприятие'), ('1c$предприятие', '1спредприятие')]
+[('1с$77', '1с77'), ('1c$77', '1с$77'), ('1c$77', '1с77')]
+[('html', 'html5')]
+[('mac$os', 'macos')]
+[('css', 'css3')]
+[('mssql', 'mysql')]
+[('vb', 'vba')]
+[('3dmax', '3ds$max'), ('3d$max', '3ds$max'), ('3d$max', '3dmax')]
+[('adobe$premier', 'adobe$premiere')]
+[('sql', 'tsql')]
+[('corel$draw', 'coreldraw')]
+[('compas', 'kompas')]
+[('office', 'offise')]
+[('консультант$плюс', 'консультантплюс')]
+[('водительский$удостоверение$категория$$', 'водительский$удостоверение$категория$b')]
+[('mathlab', 'matlab')]
+[('пк$$уверенный$пользователь', 'пк$уверенный$пользователь')]
+[('fine$reader', 'finereader')]
+[('power$point', 'powerpoint')]
+[('компас$3d', 'компас3d')]
+[('solid$works', 'solidworks')]
+[('auto$cad', 'autocad')]
+[('html', 'html5')]
+[('open$office', 'openoffice')]
)
NOT_SYNONYMS = (
[('1с8', 'c$'), ('c$', 'r'), ('1с82', 'pr'),
 ('1c', 'с'), ('pc', 'r'), ('c', 'pc'),
 ('pc', 'pr'), ('1с82', 'r'),
 ('1с', 'c'), ('1c', 'pc'), ('$1с', 'с'),
 ('c', 'pr'), ('1c', 'pr'), ('$1с', 'c'),
 ('1с$8', 'c'), ('1c', 'c'), ('1с82', 'c$'),
 ('c', 'r'), ('$1с', 'r'), ('pr', 'r'), ('1с$8', 'с'),
 ('1с8', 'pc'), ('1с82', 'pc'), ('$1с', 'pr'), ('1с8', 'r'), ('1с$82', 'c$'),
 ('1с', 'pr'), ('r', 'с'), ('1с', 'с'), ('1с$8', 'c$'), ('$1с', 'pc'), ('1с8', 'с'),
 ('1с$8', 'r'), ('1с', 'pc'), ('$1с', 'c$'), ('1с$82', 'r'), ('1с$8', 'pc'), ('1c', 'c$'),
 ('1с82', 'c'), ('1c', 'r'), ('1с$82', 'c'), ('1с8', 'c'), ('c$', 'pc'), ('1с', 'r'), ('pr', 'с'),
 ('1с$82', 'pc'), ('1с', 'c$'), ('c$', 'pr'), ('1с$8', 'pr'), ('pc', 'с'), ('1с$82', 'с'), ('1с8', 'pr'),
 ('1с$82', 'pr'), ('1с82', 'с')]
+[('пк', 'тз'), ('пк', 'пфр'), ('пк', 'тз'), ('тз', 'тз'), ('пфр', 'тз')]
+[('dns', 'dos'), ('dfd', 'dfs'), ('dos', 'ios'), ('dns', 'ios'), ('iis', 'ios'), ('dfs', 'ios'), ('dos', 'iis'), ('dns', 'iis'), ('dfd', 'dns'), ('dfs', 'dos'), ('dfd', 'iis'), ('dfs', 'dns'), ('dfs', 'iis'), ('dfd', 'ios'), ('dfd', 'dos')]
+[('php', 'xp'), ('hp', 'php'), ('hp', 'xp')]
+[('cms', 'css'), ('css3', 'wms'), ('css', 'wms'), ('cms', 'css3'), ('cms', 'wms')]
)
def lev_metric(x, y):
    i, j = int(x[0]), int(y[0])     # extract indices
    a, b = (data[i], data[j]) if data[i] < data[j] else (data[j], data[i])
    if (a, b) in SYNONYMS:
        return 0
    if (a, b) in NOT_SYNONYMS:
        return 100
    return levenshtein(a, b)

In [11]:
data = np.array([i[0] for i in c if i[1] > 100])

In [12]:
X = np.arange(len(data)).reshape(-1, 1)

In [13]:
res = dbscan(X, metric=lev_metric, eps=1, min_samples=1)

# переходим к построению графа

In [14]:
for i in skills:
    s = []
    for j in skills[i]:
        curr_skill = []
        for word in j:
            word = ''.join([c for c in word if str.isalnum(c)])
            if word not in MY_STOPWORDS and word in data:
                curr_skill.append(word)
        if curr_skill:
            s.append(' '.join(curr_skill))
    skills[i] = ' '.join(s)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
a = list(skills.values())

In [17]:
a = CountVectorizer(token_pattern='',
                    ngram_range=(1,1),
                    tokenizer=lambda x:str.split(x, ' '),
                    binary=True,
                    analyzer='word',
                    preprocessor=lambda x: x).fit_transform(a)

In [ ]:
b = a.dot(a.T)

### Построение графа

In [22]:
from itertools import combinations

In [19]:
data = {} # кандидат + список скилов
for name, skills_list in skills.items():
    curr_skill = []
    for s in skills_list:
        curr_skill.append(' '.join(s))
    data[name] = curr_skill

In [20]:
# to do: добавить weights так, чтобы частотные слова учитывались с меньшим весом
def metrics(d1, d2):
    try:
        return 1 - len(set(d1) & set(d2)) / len(set(d1) | set(d2))
    except:
        return 1

In [23]:
pair_dist = [] # тройка id-id-расстояние
for pair in tqdm(list(combinations(list(data.keys())[:5000], 2))):
    m = metrics(data[pair[0]], data[pair[1]])
    if m < 0.8:
        pair_dist.append((pair[0], pair[1], m))

100%|██████████| 12497500/12497500 [03:43<00:00, 55924.52it/s]


In [ ]:
import node2vec
import networkx as nx
G = nx.Graph()
G.add_weighted_edges_from(pair_dist)

In [ ]:
mm = node2vec.Node2Vec(G)

Computing transition probabilities:   0%|          | 15/4352 [08:38<43:46:12, 36.33s/it]